In [ ]:
import os
import sys
import re
import time
import matplotlib
import matplotlib.pyplot as plt
import random

parentPath = os.path.abspath("../..")
if parentPath not in sys.path:
    sys.path.insert(0, parentPath)

import model as modellib
import visualize
from model import log

%matplotlib inline 

# Root directory of the project
ROOT_DIR = parentPath

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

SCENENN_DIR = "/external_datasets/SceneNet_RGBD"

# import SceneNet
from dataset import *

def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
# Validation dataset
dataset_test = Dataset()
dataset_test.load("validation")
dataset_test.prepare()

## Detection

In [ ]:
class InferenceConfig(Config):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", config=config, model_dir=MODEL_DIR)
# model.load_weights(model.find_last()[1], by_name=True)
# model.load_weights(os.path.join(MODEL_DIR, "seg_scenenet20171121T1912/mask_rcnn_seg_scenenet_0624.h5"), by_name=True)
model.load_weights(os.path.join(MODEL_DIR, "seg_scenenet_classes20171121T1912/mask_rcnn_seg_scenenet_classes_0628.h5"), by_name=True)

In [ ]:
import colorsys
import cv2

# Test on a random image
image_id = random.choice(dataset_test.image_ids)
# image_id = 58992
image_id = 18190
# image_id = 216203
# image_id = 10926
print(image_id)

REND_DIR = "rend_2"

hsv = [(float(i) / 14, 1, 0.7) for i in range(14)]
colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))

for i in range(image_id, image_id + 15):
    original_image, image_meta, class_ids, bbox, mask=\
        modellib.load_image_gt(dataset_test, config,
                               i, use_mini_mask=False)
    image = original_image[:, :, 0:3]

    masked_img = visualize.display_instances(image, bbox, mask, class_ids, dataset_test.class_names, figsize=(8, 8), colors=colors)
    masked_img = cv2.cvtColor(masked_img, cv2.COLOR_BGR2RGB)
    cv2.imwrite(os.path.join(REND_DIR, dataset_test.image_info[i]['file_name']) + '.jpg', masked_img)
    
    r = model.detect([original_image])[0]
    
    masked_img_det = visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                    dataset_test.class_names, r['scores'], ax=get_ax(), colors=colors)
    masked_img_det = cv2.cvtColor(masked_img_det, cv2.COLOR_BGR2RGB)
    cv2.imwrite(os.path.join(REND_DIR, "det_" + dataset_test.image_info[i]['file_name']) + '.jpg', masked_img_det)